In [1]:
import pandas as pd
import pandas_ta as ta
import random
from collections import deque
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization, Flatten, Activation
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import ModelCheckpoint, ModelCheckpoint
import time
import statistics
from sklearn import preprocessing
from matplotlib import pyplot as plt
from IPython.display import clear_output
import os.path
import yfinance as yf

In [2]:
ticker = "TNA"

In [3]:
def getDB(ticker):
    tick = ticker
    # Load data
    data = yf.Ticker(tick)
    df = data.history(period="1y", interval="1d")
#     df = data.history(start="2018-12-01", end="2020-03-01")
#     start="2017-01-01", end="2017-04-30"
    
    # add data points
    df['close_per1'] = df.ta.percent_return(1)*100
    df['sma10'] = df.ta.sma(length=10)
    df['williams'] = df.ta.willr()


    df = df[[
            'open','close','sma10','williams','close_per1'
            ]]

    df = df.dropna()
    
    return df

In [4]:
ticker = "TNA"
df = getDB(ticker)

In [18]:
returnP1 = []; returnP2 = []
# setting the outputs in the df
for i in range(len(df)-1):
    if df.iloc[i]['close'] < df.iloc[i+1]['open']:
#         print("close date: %s" % (df.index[i]))
#         print("open date: %s" % (df.index[i+1]))
#         print()

        returnP1.append(df.iloc[i+1]['close_per1'])
    
        if df.iloc[i+1]['close_per1'] < 0:
            returnP2.append(-0.10)
        else:
            returnP2.append(df.iloc[i+1]['close_per1'])
#             returnP2.append(100*(df.iloc[i+1]['open']-df.iloc[i]['close'])/abs(df.iloc[i]['close']))
            
avgP1 = np.average(returnP1)
avgP2 = np.average(returnP2)

In [19]:
print("Number of trade days: %d" % (len(df)))
print("Number of trades: %d" % (len(returnP2)))
print("Percentage average when price opens higher than previous close: %f" % (avgP1))
print("Average return if loss limit is 0.1 percent: %f" % (avgP2))

Number of trade days: 239
Number of trades: 138
Percentage average when price opens higher than previous close: 1.668737
Average return if loss limit is 0.1 percent: 2.395377


In [12]:
print(100*(df.iloc[i]['close']-df.iloc[i]['open'])/abs(df.iloc[i]['open']))
print(df.iloc[i]['open'])
print(df.iloc[i]['close'])

-0.04696045478285022
85.18000030517578
85.13999938964844


In [8]:
import alpaca_trade_api as tradeapi
import requests
import time
import numpy as np
from datetime import datetime, timedelta
from pytz import timezone

BASE_URL = 'https://paper-api.alpaca.markets'
PUB_KEY = 'PKAX56PAOU4925CJJ9BP'
SEC_KEY = 'mHmA1VzpYPQ4sgFbBarM24rfUxUwXTJDfJXkSlWh'
api = tradeapi.REST(key_id= PUB_KEY, secret_key=SEC_KEY, base_url=BASE_URL)

In [17]:

symb = "FAS"


market_data = api.get_barset(symb, 'day', limit=5) # Get one bar object for each of the past 5 minutes

close_list = [] # This array will store all the closing prices from the last 5 minutes
for bar in market_data[symb]:
    close_list.append(bar.c) # bar.c is the closing price of that bar's time interval

close_list = np.array(close_list, dtype=np.float64) # Convert to numpy array
ma = np.mean(close_list)
last_price = close_list[4] # Most recent closing price

# print("Moving Average: " + str(ma))
# print("Last Price: " + str(last_price))

print(market_data['FAS'][4].c)

134.785


In [30]:
tsla = api.get_barset('FAS', '1Min', limit=10)
print(tsla)

{'FAS': [Bar({'c': 136.03, 'h': 136.03, 'l': 135.955, 'o': 136.02, 't': 1641223140, 'v': 300}), Bar({'c': 135.56, 'h': 135.56, 'l': 135.56, 'o': 135.56, 't': 1641223980, 'v': 100}), Bar({'c': 135.51, 'h': 135.51, 'l': 135.51, 'o': 135.51, 't': 1641225180, 'v': 100}), Bar({'c': 135.49, 'h': 135.49, 'l': 135.49, 'o': 135.49, 't': 1641225420, 'v': 100}), Bar({   'c': 135.105,
    'h': 135.105,
    'l': 135.105,
    'o': 135.105,
    't': 1641225600,
    'v': 100}), Bar({   'c': 134.535,
    'h': 134.535,
    'l': 134.535,
    'o': 134.535,
    't': 1641226620,
    'v': 170}), Bar({'c': 134.36, 'h': 134.36, 'l': 134.36, 'o': 134.36, 't': 1641226800, 'v': 100}), Bar({'c': 134.98, 'h': 134.98, 'l': 134.98, 'o': 134.98, 't': 1641227100, 'v': 100}), Bar({'c': 134.15, 'h': 134.15, 'l': 133.85, 'o': 133.85, 't': 1641228000, 'v': 500}), Bar({'c': 135.11, 'h': 135.11, 'l': 135.11, 'o': 135.11, 't': 1641229680, 'v': 100})]}


In [12]:

# Get daily price data for AAPL over the last 5 trading days.
barset = api.get_barset('AAPL', '1min', limit=5)
aapl_bars = barset['AAPL'][-1].c

# See how much AAPL moved in that timeframe.
week_open = aapl_bars[0].o
week_close = aapl_bars[-1].c
percent_change = (week_close - week_open) / week_open * 100
print('AAPL moved {}% over the last 5 days'.format(percent_change))

APIError: 1min is not a supported timeframe

In [17]:
api.get_quotes("AAPL", "2021-06-08", "2021-06-08", limit=10)

[Quote({   'ap': 129.8,
     'as': 1,
     'ax': 'P',
     'bp': 123.8,
     'bs': 1,
     'bx': 'P',
     'c': ['?'],
     't': '2021-06-08T08:00:00.0828Z',
     'z': 'C'}),
 Quote({   'ap': 143,
     'as': 1,
     'ax': 'P',
     'bp': 120,
     'bs': 1,
     'bx': 'P',
     'c': ['?'],
     't': '2021-06-08T08:00:00.0828Z',
     'z': 'C'}),
 Quote({   'ap': 129.7,
     'as': 1,
     'ax': 'P',
     'bp': 124.2,
     'bs': 1,
     'bx': 'P',
     'c': ['?'],
     't': '2021-06-08T08:00:00.0828Z',
     'z': 'C'}),
 Quote({   'ap': 143,
     'as': 1,
     'ax': 'P',
     'bp': 0,
     'bs': 0,
     'bx': 'T',
     'c': ['?'],
     't': '2021-06-08T08:00:00.0828Z',
     'z': 'C'}),
 Quote({   'ap': 143,
     'as': 1,
     'ax': 'P',
     'bp': 116.5,
     'bs': 1,
     'bx': 'P',
     'c': ['?'],
     't': '2021-06-08T08:00:00.0828Z',
     'z': 'C'}),
 Quote({   'ap': 129.7,
     'as': 1,
     'ax': 'P',
     'bp': 123.8,
     'bs': 1,
     'bx': 'P',
     'c': ['?'],
     't': '2021-06

In [21]:

# Get our account information.
account = api.get_account()

# Check if our account is restricted from trading.
if account.trading_blocked:
    print('Account is currently restricted from trading.')

# Check how much money we can use to open new positions.
print('${} is available as buying power.'.format(account.buying_power))

$15783.864 is available as buying power.


In [22]:

# Get a list of all active assets.
active_assets = api.list_assets(status='active')

# Filter the assets down to just those on NASDAQ.
nasdaq_assets = [a for a in active_assets if a.exchange == 'NASDAQ']
print("Number of tradable tickers in the NASDAQ: {}.".format(len(nasdaq_assets)))

Number of tradable tickers in the NASDAQ: 4805.


In [23]:

# Check if the market is open now.
clock = api.get_clock()
print('The market is {}'.format('open.' if clock.is_open else 'closed.'))

# Check when the market was open on Dec. 1, 2018
DATE = '2018-12-01'
calendar = api.get_calendar(start=DATE, end=DATE)[0]
print('The market opened at {} and closed at {} on {}.'.format(
    calendar.open,
    calendar.close,
    DATE
))


The market is open.
The market opened at 09:30:00 and closed at 16:00:00 on 2018-12-01.


In [24]:
from datetime import date

today = date.today()
print("Today's date:", today)


Today's date: 2022-01-03


In [8]:
print(DATE)
print(today)
print(type(today))

2018-12-01
2022-01-02
<class 'datetime.date'>


In [13]:
ticker = "FAS"

trading = True; days = 1
# making a loop that will last for 3 days
while trading:
    print("Day {}.".format(days))
    # waiting for the day to change
    todayDate = date.today() # setting todays date
    times = 0 # for testing reasons it will loop 3 times.
    while date.today() == todayDate: # if it is still the same day it will stay up here in the code
        print('still the same day')
        times += 1
        if times == 3:
            todayDate = 'not today'
        time.sleep(.25) # sleeping for 0.5 second so it isn't freaking out
        
    print('the day changed')

    # Get daily price data for AAPL over the last 5 trading days.
    barset = api.get_barset(ticker, 'day', limit=5)
    open_price = barset[ticker][-1].c
    print('previous day close {}.'.format(open_price))


    clock = api.get_clock()
    times = 0
    while not clock.is_open:
        times += 1
        print('still isn''t open')
        if times == 3:
            break
        
    print('the market opened up')

        
    days += 1
    if days == 4:
        trading = False
    
    print()


Day 1.
still the same day
still the same day
still the same day
the day changed
previous day close 130.7.
still isnt open
still isnt open
still isnt open
the market opened up

Day 2.
still the same day
still the same day
still the same day
the day changed
previous day close 130.7.
still isnt open
still isnt open
still isnt open
the market opened up

Day 3.
still the same day
still the same day
still the same day
the day changed
previous day close 130.7.
still isnt open
still isnt open
still isnt open
the market opened up



In [73]:
clock.is_open = True


In [74]:
clock.is_open 

True

In [21]:
import datetime
ct = datetime.datetime.now()
print("current time:-", ct)


current time:- 2022-01-02 23:34:47.492965


In [22]:
## the script will loop here untill the market opens ##
clock = api.get_clock() # gets if the market is open
while not clock.is_open:
    ct = datetime.datetime.now()
print("The market opened at this time: {}.".format(ct))

KeyboardInterrupt: 

In [23]:
print("The market opened at this time: {}.".format(ct))

The market opened at this time: 2022-01-02 23:36:05.355848.


In [24]:
clock = api.get_clock() # gets if the market is open

print()
print("The market opened up at timestamp: {}.".format(clock.timestamp))


The market opened up at timestamp: 2022-01-03 00:09:02.174095938-05:00.


In [25]:
clock.timestamp.minute

9

In [27]:
def convertTS(timestamp):
    t = timestamp.to_pydatetime()
    s = t.strftime('%Y-%m-%d')
    return s
lq = api.get_last_quote("AAPL")
# print(lq)
ts = convertTS(lq.timestamp)
# info = lq.timestamp
# t = info.to_pydatetime()
# s = t.strftime('%Y-%m-%d')
print(ts)
print(lq)

2022-01-03
Quote({   'askexchange': 15,
    'askprice': 183.45,
    'asksize': 1,
    'bidexchange': 15,
    'bidprice': 181.33,
    'bidsize': 4,
    'timestamp': 1641229590425260936})


<ipython-input-27-a3efe4c91f3f>:7: UserWarning: Discarding nonzero nanoseconds in conversion
  ts = convertTS(lq.timestamp)
